# Manual PSBT CoinJoin Workflow

![coinjoin workflow](https://raw.githubusercontent.com/bitcoin/bips/refs/heads/master/bip-0174/coinjoin-workflow.svg "Coinjoin Workflow")

In [8]:
import bits.keys

alice_key = bits.keys.key()
alice_pubkey = bits.keys.pub(alice_key, compressed=True)
alice_p2pkh_addr = bits.to_bitcoin_address(bits.crypto.hash160(alice_pubkey), network="regtest")
print(alice_p2pkh_addr)

b'mkJEaG9s5xKMdFje9zi47bKdF4TrFRMEzP'


In [9]:
# send some bitcoin to alice
# we're running bitcoind in regtest mode and we can mine to get the block reward sent to alice's address
# with the following CLI command (be sure to configure rpc settings in the config):

# bits mine --limit 101 --recv-addr mkJEaG9s5xKMdFje9zi47bKdF4TrFRMEzP

# then find info on your utxos with:
# bits rpc scantxoutset start '["addr(mkJEaG9s5xKMdFje9zi47bKdF4TrFRMEzP)"]

In [10]:
# {
#     "txid": "654355d7d26943b214cb5bfdd114868a7bf3f25bd01013b74e444cc891a27efd",
#     "vout": 0,
#     "scriptPubKey": "76a9143470cf91998e238b2ca26dceacae924dba9b762c88ac",
#     "desc": "addr(mkJEaG9s5xKMdFje9zi47bKdF4TrFRMEzP)#4exy2s7f",
#     "amount": 50,
#     "coinbase": true,
#     "height": 34
# }

import bits.tx

outpoint = bits.tx.outpoint(bytes.fromhex("654355d7d26943b214cb5bfdd114868a7bf3f25bd01013b74e444cc891a27efd")[::-1], 0)
script_sig = b""    # empty for now
txins = [bits.tx.txin(outpoint, script_sig)]

In [11]:
# generate another address to send to 

darwin_key = bits.keys.key()
darwin_pubkey = bits.keys.pub(darwin_key, compressed=True)
darwin_p2pkh_addr = bits.to_bitcoin_address(bits.crypto.hash160(darwin_pubkey), network="regtest")
print(darwin_p2pkh_addr)

b'mn8WNs1F5tKTQ2AHZMSjhdN7XTg2msTRLe'


In [19]:
import bits.script
import bits.base58

txouts = [
    bits.tx.txout(
        int(2500e6),     # 25 BTC
        bits.script.p2pkh_script_pubkey(
            bits.base58.base58check_decode(darwin_p2pkh_addr)[1:]   # == bits.crypto.hash160(darwin_pubkey)
        )
    )
]

tx = bits.tx.tx(txins, txouts)
print(tx.hex())

0100000001fd7ea291c84c444eb71310d05bf2f37b8a8614d1fd5bcb14b24369d2d75543650000000000ffffffff0100f90295000000001976a914488a1846eac8f925ce01ea941a852ef5a461032a88ac00000000


In [18]:
# use
# a) bits rpc testmempoolaccept, or 
# b) bits rpc decoderawtransaction
# to somewhat verify the raw transaction
# you may need to mine more blocks to spend coinbase txin
# note: bits rpc is basically a passthrough for bitcoin-cli but does have the advantage of leveraging the bits config with rpc settings etc


# OK...
# now to form PSBT to send to Bob

In [ ]:

import bits.tx

psbt = bits.tx.create_psbt(tx, b64encode=True)
print(psbt.decode("utf8"))

cHNidP8BAFUCAAAAAf1+opHITEROtxMQ0Fvy83uKhhTR/VvLFLJDadLXVUNlAAAAAAD9////AQDyBSoBAAAAGXapFEiKGEbqyPklzgHqlBqFLvWkYQMqiKwAAAAAAAAA

cHNidP8BAFUBAAAAAf1+opHITEROtxMQ0Fvy83uKhhTR/VvLFLJDadLXVUNlAAAAAAD/////AQD5ApUAAAAAGXapFEiKGEbqyPklzgHqlBqFLvWkYQMqiKwAAAAAAAAA


In [46]:
# ok, now hand to bob
import json
from bits.bips import bip174

parsed_psbt = bip174.parse_psbt(psbt, b64decode=True)
print(json.dumps(parsed_psbt, indent=2))


{
  "psbt_version": 0,
  "global_map": [
    {
      "keytype": 0,
      "keydata": "",
      "valuedata": "0100000001fd7ea291c84c444eb71310d05bf2f37b8a8614d1fd5bcb14b24369d2d75543650000000000ffffffff0100f90295000000001976a914488a1846eac8f925ce01ea941a852ef5a461032a88ac00000000"
    }
  ],
  "input_map": [],
  "output_map": []
}
